In [ ]:
from torch_geometric.datasets import Planetoid

import torch
import numpy as np
from tqdm.notebook import tqdm
from torch_geometric.nn import Linear, GATv2Conv
import torch.nn.functional as F

In [ ]:
ds = Planetoid(root="/tmp/Cora", name="Cora")[0]

In [ ]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads):

        super().__init__()
        self.conv1 = GATv2Conv(in_channels, hidden_channels, heads, edge_dim=3, dropout=0.2)
        self.conv2 = GATv2Conv(hidden_channels * heads, hidden_channels, heads, edge_dim=3, dropout=0.2)
        self.lin1 = Linear(hidden_channels * heads, 32)
        self.lin2 = Linear(32, 8)
        self.linout = Linear(8, out_channels)

    def forward(self, x, edge_index, edge_attr=None):
        x = F.relu(self.conv1(x, edge_index, edge_attr))
        x = F.relu(self.conv2(x, edge_index, edge_attr))
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(self.lin2(x))
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.linout(x)
        x = F.log_softmax(x)

        return x

In [ ]:
model = GAT(
    in_channels=ds.x.shape[1],
    hidden_channels=32,
    heads=4,
    out_channels=7
)

sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-3)

model.train()
for epoch in tqdm(range(10000), total=10000):
    optimizer.zero_grad()
    out = model(x=ds.x, edge_index=ds.edge_index)
    loss = F.nll_loss(out[ds.train_mask], ds.y[ds.train_mask])
    print(loss)
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
pred = model(x=ds.x, edge_index=ds.edge_index)
pred_proba = torch.exp(pred[ds.test_mask])
pred = torch.argmax(pred_proba, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(accuracy_score(ds.y[ds.test_mask], pred))
confusion_matrix(ds.y[ds.test_mask], pred)